<a href="https://colab.research.google.com/github/ysooch0819/AI16-Projects/blob/main/Section2_project%EC%A4%80%EB%B9%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests
import time
import json
from pandas import json_normalize

In [3]:
import re

In [200]:
# api_key = getAPIkey()
api_key = 'RGAPI-e529c646-303f-44f7-84cc-05bc785b6bb5'

REGION_SET= 'kr'
tier = 'challenger'
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com"
}
url= 'https://kr.api.riotgames.com/tft/league/v1/challenger?api_key=' + api_key

req = requests.get(url)

In [201]:
len(req.json()['entries'])

300

In [305]:
challenger_sample = pd.DataFrame(req.json()).reset_index()
challenger_sample.to_csv('sample.csv', index=False, encoding='cp949')

In [10]:
challenger_sample = pd.read_csv('sample.csv', encoding='cp949').set_index('index')
challenger_sample

,tier,leagueId,queue,name,entries
index,,,,,
0,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'Wq6S0VXK-5RxqhCIoj43jaY7yap4wl...
1,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'kRsaMVAuKMJ98pVtiiHuIliwEpJiGq...
2,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'LZYr5TN16DmatrYfG5nHRx7xjLLD-8...
3,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'EevAodEB1ogihddloTtNQmr237TbPY...
4,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'oarzOITC6LUddm097p5zMhpXFT2zeR...
...,...,...,...,...,...
295,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'alMRPqSjnkp4OII6-ZBbCdqdt3jXuh...
296,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'UiV0CKlJ-tHX7LVwQ2Z95nxdox8OMn...
297,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'cSXx2KKA1LyGEDbS1Od8GH_Zwoduvf...


In [11]:
challenger_entries = pd.DataFrame(dict(challenger_sample['entries'])).T
challenger_entries

ValueError: ignored

In [309]:
challenger_sample = pd.concat([challenger_sample, challenger_entries], axis=1)

In [313]:
challenger_sample.head()

,index,tier,leagueId,queue,name,entries,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,0,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'Wq6S0VXK-5RxqhCIoj43jaY7yap4wl...,Wq6S0VXK-5RxqhCIoj43jaY7yap4wlOluyY2OKT35Gzvng,관자빵,1026,I,202,170,False,False,False,True
1,1,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'kRsaMVAuKMJ98pVtiiHuIliwEpJiGq...,kRsaMVAuKMJ98pVtiiHuIliwEpJiGqSDRVhfrS50TdPkwt...,Inv Artist,1338,I,428,273,True,False,False,True
2,2,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'LZYr5TN16DmatrYfG5nHRx7xjLLD-8...,LZYr5TN16DmatrYfG5nHRx7xjLLD-8dmSTvgdb5Uk-88CcY,fanny1,1088,I,377,340,False,False,True,True
3,3,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'EevAodEB1ogihddloTtNQmr237TbPY...,EevAodEB1ogihddloTtNQmr237TbPYwIGPedgVJ4qum96X...,엠 돈,1055,I,299,260,False,False,False,False
4,4,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'oarzOITC6LUddm097p5zMhpXFT2zeR...,oarzOITC6LUddm097p5zMhpXFT2zeRSX4ZHJ77mv1zcfor0,철회랜드,1025,I,196,158,False,False,True,True


In [312]:
challenger_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         300 non-null    int64 
 1   tier          300 non-null    object
 2   leagueId      300 non-null    object
 3   queue         300 non-null    object
 4   name          300 non-null    object
 5   entries       300 non-null    object
 6   summonerId    300 non-null    object
 7   summonerName  300 non-null    object
 8   leaguePoints  300 non-null    object
 9   rank          300 non-null    object
 10  wins          300 non-null    object
 11  losses        300 non-null    object
 12  veteran       300 non-null    object
 13  inactive      300 non-null    object
 14  freshBlood    300 non-null    object
 15  hotStreak     300 non-null    object
dtypes: int64(1), object(15)
memory usage: 37.6+ KB


In [12]:
df = challenger_sample.copy()

In [13]:
df

,tier,leagueId,queue,name,entries
index,,,,,
0,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'Wq6S0VXK-5RxqhCIoj43jaY7yap4wl...
1,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'kRsaMVAuKMJ98pVtiiHuIliwEpJiGq...
2,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'LZYr5TN16DmatrYfG5nHRx7xjLLD-8...
3,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'EevAodEB1ogihddloTtNQmr237TbPY...
4,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'oarzOITC6LUddm097p5zMhpXFT2zeR...
...,...,...,...,...,...
295,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'alMRPqSjnkp4OII6-ZBbCdqdt3jXuh...
296,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'UiV0CKlJ-tHX7LVwQ2Z95nxdox8OMn...
297,CHALLENGER,946274f2-16b6-376e-8d60-e07ef8f06a7f,RANKED_TFT,Orianna's Dragons,{'summonerId': 'cSXx2KKA1LyGEDbS1Od8GH_Zwoduvf...


In [316]:
# 필요없는 컬럼 제거: 'index', 'leagueId', 'queue', 'name', 'eentries', 'rank'
drop_cols = ['index', 'leagueId', 'queue', 'name', 'entries', 'rank']
df = df.drop(drop_cols, axis=1, errors='ignore')

In [318]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          300 non-null    object
 1   entries       300 non-null    object
 2   summonerId    300 non-null    object
 3   summonerName  300 non-null    object
 4   leaguePoints  300 non-null    object
 5   wins          300 non-null    object
 6   losses        300 non-null    object
 7   veteran       300 non-null    object
 8   inactive      300 non-null    object
 9   freshBlood    300 non-null    object
 10  hotStreak     300 non-null    object
dtypes: object(11)
memory usage: 25.9+ KB


In [319]:
df.to_csv('sample_user.csv', index=False, encoding='cp949')

In [320]:
## entries 딕셔너리에 들어있던 특성들 설명
# veteran : 현재 리그에서 최소 100게임 이상 플레이한 플레이어
# inactive : 휴면계정
# freshBlood : 최근에 리그에 합류한 새내기
# hotStreak : 연승 중인 플레이어

In [353]:
df.summonerName.nunique()

300

In [354]:
match_id = pd.DataFrame()

for i in range(0, len(df)-1):
    url_puuid = 'https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/'\
                +df.summonerName[i]+'?api_key=' + api_key
    url_match = 'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/'\
                +requests.get(url_puuid).json()['puuid']+'/ids?start=0&count=200&api_key=' +api_key
    req_match = requests.get(url_match)
    match_id.append(req_match.json())


KeyError: ignored

In [361]:
match_id = pd.DataFrame()

In [427]:
for i in range(284, len(df)-1):
    url_puuid = 'https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/'\
                  +df.summonerName[i]+'?api_key=' + api_key
    url_match = 'https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/'\
                  +requests.get(url_puuid).json()['puuid']+'/ids?start=0&count=200&api_key=' +api_key
    req_match = requests.get(url_match)
    match_id = match_id.append(req_match.json())

In [428]:
print('match_id 개수: ', len(match_id))
match_id

match_id 개수:  59598


,0
0,KR_6233363333
1,KR_6233290882
2,KR_6232501927
3,KR_6232409409
4,KR_6232324593
...,...
195,KR_6082653943
196,KR_6082591610
197,KR_6082557590
198,KR_6082406064


In [433]:
match_id = match_id.drop_duplicates()

In [438]:
len(match_id)
match_id.rename(columns = {0: 'match_id'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [439]:
match_id.to_csv('match_ID.csv', index=False, encoding='cp949')

In [14]:
match_id = pd.read_csv('match_ID.csv', encoding='cp949')
match_id

,match_id
0,KR_6233363333
1,KR_6233290882
2,KR_6232501927
3,KR_6232409409
4,KR_6232324593
...,...
34773,KR_6082653943
34774,KR_6082591610
34775,KR_6082557590
34776,KR_6082406064


In [441]:
match_id # 34778 개의 중복되지 않은 챌린저티어 매치

,match_id
0,KR_6233363333
1,KR_6233290882
2,KR_6232501927
3,KR_6232409409
4,KR_6232324593
...,...
195,KR_6082653943
196,KR_6082591610
197,KR_6082557590
198,KR_6082406064


In [15]:
df = match_id.copy()

In [16]:
df.iloc[0]['match_id']

'KR_6233363333'

In [199]:
len(df)

68901

In [41]:
# metadf = pd.DataFrame()

In [49]:
import csv
f = open('rawdata.csv', encoding='cp949')
reader = csv.reader(f)
csv_list = []
for l in reader:
    csv_list.append(l)
f.close()
metadf = pd.DataFrame(csv_list)
metadf

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,augments,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,...,None,None,None,None,None,None,None,None,None,None
1,"['TFT6_Augment_JeweledLotus', 'TFT7_Augment_Pa...",{'content_ID': 'c4abff3d-3a21-48c4-940d-ad54b3...,50,38,8,2,2,gihX7aaLSt3urJK1lTv3p87hA4aLaNhqwr030xceD9-pn1...,2311.57251,183,...,None,None,None,None,None,None,None,None,None,None
2,"['TFT6_Augment_TradeSector', 'TFT6_Augment_Fir...",{'content_ID': '6054d70b-e401-4bec-b350-f26865...,3,27,8,7,0,EDzYOAvlgpa2Mx5utJGl-7rMEysGqpEV1C82lO-QsiUXOd...,1690.70752,38,...,None,None,None,None,None,None,None,None,None,None
3,"['TFT6_Augment_TomeOfTraits1', 'TFT7_Augment_P...",{'content_ID': '130a03b8-8dfe-47f8-9fa1-02d4ad...,1,28,8,5,0,n6fDw9WjtVHOZqIxzP4WPoGlbBtxO13OE0ZJ-IoDoc4Oi4...,1762.875732,69,...,None,None,None,None,None,None,None,None,None,None
4,"['TFT6_Augment_CelestialBlessing2', 'TFT6_Augm...",{'content_ID': 'ee3fda5c-1956-44fe-a4a4-377483...,1,31,8,3,0,0Dy-7-nO1q539sOfPaFZuGyfmVI89QKyYyBcQqBg5UZSZh...,1878.452148,121,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68910,"['TFT6_Augment_TrueTwos', 'TFT7_Augment_Mikael...",{'content_ID': 'fab64a15-f7d3-4313-9f7c-a88817...,1,31,8,5,0,41ytzNMZTGjGp0f6Zim6lGk5WM023j1rDRSFw2iyydrYYv...,1837.862305,96,...,None,None,None,None,None,None,None,None,None,None
68911,"['TFT6_Augment_Ascension', 'TFT6_Augment_Secon...",{'content_ID': '8f032d75-83f7-445c-b2dd-751dc2...,0,35,8,3,0,5B72006bOGGg3uL2Up6jP_-DWVEGbzilblIs3k8F-hC-IQ...,2089.41333,89,...,None,None,None,None,None,None,None,None,None,None
68912,"['TFT6_Augment_Ascension', 'TFT6_Augment_Porta...",{'content_ID': '8c82996d-a343-4319-9a9a-9544f8...,7,41,8,2,2,bX6jR_PKgpD_JceDQCWgoiY72t9JpwgpRhv0r7UUJmLZil...,2415.433105,169,...,None,None,None,None,None,None,None,None,None,None
68913,"['TFT7_Augment_AxiomArc2', 'TFT6_Augment_Elect...",{'content_ID': '54d41338-c532-4522-8bf4-521979...,11,26,8,8,0,MOxPQIoI1ucSkutzvNx8iabNwTEkVn1Nn44MWypYl8hPcS...,1526.32251,30,...,None,None,None,None,None,None,None,None,None,None


In [50]:
metadf.rename(columns={
    0 : 'augments',
    1 : 'companion',
    2 : 'gold_left',
    3 : 'last_round',
    4 : 'level',
    5 : 'placement',
    6 : 'players_eliminated',
    7 : 'puuid',
    8 : 'time_eliminated',
    9 : 'total_damage_to_players',
    10 : 'traits',
    11 : 'units'
}, inplace=True)

In [56]:
len(metadf) - metadf.duplicated().sum()

53508

In [65]:
features = ['augments','companion','gold_left','last_round','level','placement','players_eliminated','puuid','time_eliminated','total_damage_to_players','traits','units']
metadf = metadf[features]

In [67]:
metadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68915 entries, 0 to 68914
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   augments                 68915 non-null  object
 1   companion                68915 non-null  object
 2   gold_left                68914 non-null  object
 3   last_round               68909 non-null  object
 4   level                    68907 non-null  object
 5   placement                68907 non-null  object
 6   players_eliminated       68907 non-null  object
 7   puuid                    68907 non-null  object
 8   time_eliminated          68907 non-null  object
 9   total_damage_to_players  68907 non-null  object
 10  traits                   68907 non-null  object
 11  units                    68906 non-null  object
dtypes: object(12)
memory usage: 6.3+ MB


In [76]:
metadf = metadf.drop(0).reset_index(drop=True)

In [86]:
metadf = metadf.dropna().reset_index(drop=True)

In [89]:
metadf.head()

,augments,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,"['TFT6_Augment_JeweledLotus', 'TFT7_Augment_Pa...",{'content_ID': 'c4abff3d-3a21-48c4-940d-ad54b3...,50,38,8,2,2,gihX7aaLSt3urJK1lTv3p87hA4aLaNhqwr030xceD9-pn1...,2311.57251,183,"[{'name': 'Set7_Assassin', 'num_units': 1, 'st...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
1,"['TFT6_Augment_TradeSector', 'TFT6_Augment_Fir...",{'content_ID': '6054d70b-e401-4bec-b350-f26865...,3,27,8,7,0,EDzYOAvlgpa2Mx5utJGl-7rMEysGqpEV1C82lO-QsiUXOd...,1690.70752,38,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Rakan', 'itemNames': [..."
2,"['TFT6_Augment_TomeOfTraits1', 'TFT7_Augment_P...",{'content_ID': '130a03b8-8dfe-47f8-9fa1-02d4ad...,1,28,8,5,0,n6fDw9WjtVHOZqIxzP4WPoGlbBtxO13OE0ZJ-IoDoc4Oi4...,1762.875732,69,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Rakan', 'itemNames': [..."
3,"['TFT6_Augment_CelestialBlessing2', 'TFT6_Augm...",{'content_ID': 'ee3fda5c-1956-44fe-a4a4-377483...,1,31,8,3,0,0Dy-7-nO1q539sOfPaFZuGyfmVI89QKyYyBcQqBg5UZSZh...,1878.452148,121,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Wukong', 'itemNames': ..."
4,"['TFT7_Augment_ShimmerscaleTrait', 'TFT7_Augme...",{'content_ID': 'ee3fda5c-1956-44fe-a4a4-377483...,3,38,9,1,2,Ad8Gb6OmmJ8u00gXFfLlN-fFCS_z95Ouj-filwtAlUd1BG...,2311.57251,207,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Nasus', 'itemNames': [..."


In [101]:
metadf = metadf.drop([28437, 31689, 16768, 61423]).reset_index(drop=True)

In [102]:
metadf.placement.value_counts()

1    8697
5    8603
8    8603
4    8602
3    8600
2    8599
6    8599
7    8598
Name: placement, dtype: int64

In [111]:
metadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68901 entries, 0 to 68900
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   augments                 68901 non-null  object
 1   companion                68901 non-null  object
 2   gold_left                68901 non-null  object
 3   last_round               68901 non-null  object
 4   level                    68901 non-null  object
 5   placement                68901 non-null  int64 
 6   players_eliminated       68901 non-null  object
 7   puuid                    68901 non-null  object
 8   time_eliminated          68901 non-null  object
 9   total_damage_to_players  68901 non-null  object
 10  traits                   68901 non-null  object
 11  units                    68901 non-null  object
dtypes: int64(1), object(11)
memory usage: 6.3+ MB


In [135]:
df = metadf.drop(columns = ['companion', 'puuid'])
df

,augments,gold_left,last_round,level,placement,players_eliminated,time_eliminated,total_damage_to_players,traits,units
0,"['TFT6_Augment_JeweledLotus', 'TFT7_Augment_Pa...",50,38,8,2,2,2311.57251,183,"[{'name': 'Set7_Assassin', 'num_units': 1, 'st...","[{'character_id': 'TFT7_Vladimir', 'itemNames'..."
1,"['TFT6_Augment_TradeSector', 'TFT6_Augment_Fir...",3,27,8,7,0,1690.70752,38,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Rakan', 'itemNames': [..."
2,"['TFT6_Augment_TomeOfTraits1', 'TFT7_Augment_P...",1,28,8,5,0,1762.875732,69,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Rakan', 'itemNames': [..."
3,"['TFT6_Augment_CelestialBlessing2', 'TFT6_Augm...",1,31,8,3,0,1878.452148,121,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Wukong', 'itemNames': ..."
4,"['TFT7_Augment_ShimmerscaleTrait', 'TFT7_Augme...",3,38,9,1,2,2311.57251,207,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Nasus', 'itemNames': [..."
...,...,...,...,...,...,...,...,...,...,...
68896,"['TFT6_Augment_TomeOfTraits1', 'TFT7_Augment_F...",1,33,8,4,1,1955.89209,79,"[{'name': 'Set7_Cavalier', 'num_units': 1, 'st...","[{'character_id': 'TFT7_Twitch', 'itemNames': ..."
68897,"['TFT6_Augment_TrueTwos', 'TFT7_Augment_Mikael...",1,31,8,5,0,1837.862305,96,"[{'name': 'Set7_Cavalier', 'num_units': 1, 'st...","[{'character_id': 'TFT7_Twitch', 'itemNames': ..."
68898,"['TFT6_Augment_Ascension', 'TFT6_Augment_Secon...",0,35,8,3,0,2089.41333,89,"[{'name': 'Set7_Assassin', 'num_units': 2, 'st...","[{'character_id': 'TFT7_Qiyana', 'itemNames': ..."
68899,"['TFT6_Augment_Ascension', 'TFT6_Augment_Porta...",7,41,8,2,2,2415.433105,169,"[{'name': 'Set7_Bard', 'num_units': 1, 'style'...","[{'character_id': 'TFT7_Zyra', 'itemNames': ['..."


In [136]:
winners = df['placement'] <= 4
len(df[winners]), len(df[~winners])

(34498, 34403)

In [ ]:
raw_df = pd.DataFrame()

In [495]:
url_ingame = 'https://asia.api.riotgames.com/tft/match/v1/matches/'\
              +df.iloc[1]['match_id']+ '?api_key=' + api_key
metadf = metadf.append(pd.DataFrame(requests.get(url_ingame).json()['info']['participants']))

In [ ]:
for i in range(9708, len(df)-1):
    time.sleep(1.2)
    url_ingame = 'https://asia.api.riotgames.com/tft/match/v1/matches/'+df.iloc[i]['match_id']+ '?api_key=' + api_key
    metadf = metadf.append(pd.DataFrame(requests.get(url_ingame).json()['info']['participants']))

In [569]:
%%time
url_ingame = 'https://asia.api.riotgames.com/tft/match/v1/matches/'+df.iloc[9707]['match_id']+ '?api_key=' + api_key
pd.DataFrame(requests.get(url_ingame).json()['info']['participants'])

KeyError: ignored

In [47]:
print(len(metadf)/8)
metadf.placement.value_counts()

1572.5


1            1585
2            1571
4            1571
6            1571
8            1571
7            1570
5            1570
3            1569
placement       1
Name: placement, dtype: int64

In [48]:
metadf.tail(8)

,augments,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,...,57,58,59,60,61,62,63,64,65,66
12572,"['TFT6_Augment_PortableForge', 'TFT7_Augment_A...",{'content_ID': 'ee3fda5c-1956-44fe-a4a4-377483...,1,30,8,7,0,1o5sWId6ZRm_eUMQqBy7f9vdvQAOeiAjvckZDZOJTpN5tM...,1806.55481,102,...,None,None,None,None,None,None,None,None,None,None
12573,"['TFT7_Augment_GuildEmblem', 'TFT6_Augment_Ite...",{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,2,35,8,5,2,3vJApffY1wzCp4o5ZyhhxHIu9Pc2fkSgltJMp1MMduHivr...,2120.306885,130,...,None,None,None,None,None,None,None,None,None,None
12574,"['TFT6_Augment_JeweledLotus', 'TFT7_Augment_La...",{'content_ID': '815a37f3-2cba-487c-bda9-6f49ef...,13,37,9,3,1,9DEPnQysNaeEbh-Rbq6qRG7RJxVGHkZcxqBtV_LzkhInbj...,2232.18042,137,...,None,None,None,None,None,None,None,None,None,None
12575,"['TFT7_Augment_ShimmerscaleTrait', 'TFT6_Augme...",{'content_ID': 'e83858ae-1837-401c-8330-f063cd...,8,26,8,8,0,VS6mODKhDud20CBCtPw1hRj9kgl70DOcjfVjr6we_FSgrJ...,1574.119385,37,...,None,None,None,None,None,None,None,None,None,None
12576,"['TFT7_Augment_AxiomArc2', 'TFT7_Augment_BandO...",{'content_ID': 'c5a62aeb-1821-463b-a413-308814...,1,21,5,8,0,k-nYWDVyww6Vv9YKuMuxG_eC0qVqSTndcxQ5Pa6jr44jLX...,1299.802734,4,...,None,None,None,None,None,None,None,None,None,None
12577,"['TFT7_Augment_ShimmerscaleTrait', 'TFT6_Augme...",{'content_ID': '54d41338-c532-4522-8bf4-521979...,24,30,8,6,0,OpEBGhInVTTK3hV9x0x2dH_GJbVeOdF9I7FPvZLuFqwczE...,1824.631714,57,...,None,None,None,None,None,None,None,None,None,None
12578,"['TFT7_Augment_SwiftshotEmblem', 'TFT6_Augment...",{'content_ID': '90c30ac2-94d9-4f5a-9c78-51f651...,0,41,8,2,2,JfJyJYv79OCtj7nAnqEFke7WzJ_-Ar6PYz7G7fVfdA6gWs...,2443.937012,153,...,None,None,None,None,None,None,None,None,None,None
12579,"['TFT6_Augment_CyberneticUplink2', 'TFT6_Augme...",{'content_ID': 'b8d85a23-1946-4e31-acd5-97d7be...,1,41,9,1,1,4UlK42ZBlK-3GEmIGx3quFlgDt9Qdq7ugwboFHI0gv_yIV...,2443.937012,236,...,None,None,None,None,None,None,None,None,None,None


In [567]:
metadf.to_csv('rawdata.csv', index=False, encoding='cp949')

In [137]:
df.to_csv('test_df.csv', index=False)

In [502]:
# 'info' 안의 'participants' 특성에 해당 게임의 경기에서 8명의 전체유저들이 플레이한 기록이 들어있다.
# 관심있는것은 각 유저들의 해당 경기의 placement(최종순위)가 어떻게 되었는가이다.
# TFT의 랭크게임은 8명의 유저 중 최종 순위 1~4위까지는 tier 점수가 오르고, 5~8위는 tier 점수가 깎이는 방식으로 진행된다.
# 알고 싶은 것은 유저들이 게임 내 어떠한 선택지를 골랐을 때 tier(점수)를 쉽게 올릴 수 있을까? 것이므로, 게임별 상위 4명의 데이터와 하위 4명의 데이터를 별도로 분석한다.
# 필요하다면 Grandmaster, master 의 데이터도 소량 가져와야한다.

In [ ]:
# 안정적인 티어덱은 무엇인가?
# target = 'placement'
# 순위와 직접적으로 연결되는 특성은 배제
# 'last_round' : 유저가 생존한 최종 round 수, 1등과 2등/3등이 마지막 round를 진행할테니 상위로 갈수록 높을 수 밖에 없다.
# 'time_eliminated': 유저가 플레이한 전체 시간, 위와 동일한 이유
# 
# 'augments', 'gold_left', 'level', 'total_damage_to_players', 'traits', 'units' 를 활용해 분석해야한다.